In [24]:
import pandas as pd

file_names = ['../data/Resale Flat Prices (Based on Approval Date), 1990 - 1999.csv', 
              '../data/Resale Flat Prices (Based on Approval Date), 2000 - Feb 2012.csv', 
              '../data/Resale Flat Prices (Based on Registration Date), From Mar 2012 to Dec 2014.csv', 
              '../data/Resale Flat Prices (Based on Registration Date), From Jan 2015 to Dec 2016.csv', 
              '../data/Resale flat prices based on registration date from Jan-2017 onwards.csv']

initial_counts = []
final_counts = []

dfs = []

for file in file_names:
    df = pd.read_csv(file)
    
    initial_count = len(df)
    initial_counts.append(initial_count)
    
    df = df.dropna()
    cleaned_count = len(df)
    final_counts.append(cleaned_count)
    
    if 'remaining_lease' not in df.columns:
        df['year'] = df['month'].str[:4].astype(int)

        df['remaining_lease'] = (df['lease_commence_date'] + 99) - df['year']

        df.drop(columns=['year'], inplace=True)
    
    dfs.append(df)

merged_df = pd.concat(dfs, ignore_index=True)

total_initial = sum(initial_counts)
total_final = len(merged_df)
lost_rows = total_initial - total_final

print("Total initial rows across all tables:", total_initial)
print("Total rows after cleaning and computing remaining_lease:", total_final)
print("Total data points lost during cleaning:", lost_rows)



Total initial rows across all tables: 948962
Total rows after cleaning and computing remaining_lease: 948962
Total data points lost during cleaning: 0
